In [ ]:
import pathlib
import tempfile
import datetime
import os
import time

import numpy as np
import pandas as pd

import scipy.interpolate

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys._losslessjpeg

In [ ]:
images = list(pathlib.Path(r'S:\Temp').glob('**/*.jpg'))
# images

In [ ]:
mechanical_icom_data_path = pathlib.Path(r'\\physics-server\iComLogFiles\mechanical\4299\20200116.csv')
icom_data = pd.read_csv(mechanical_icom_data_path)
icom_data

In [ ]:
timestamps = [
    pd.Timestamp(datetime.datetime.fromtimestamp(os.path.getmtime(path)))
    for path in images
]
# timestamps

In [ ]:


pymedphys._losslessjpeg.imread()

In [ ]:
np.unique([1,1,1,2], return_counts=True)

In [ ]:
unique_timestamps, counts = np.unique(icom_data['Timestamp'], return_counts=True)
unique_timestamps = unique_timestamps.astype(np.datetime64)

In [ ]:
stamp = datetime.datetime.fromisoformat(unique_timestamps[0])
stamp + datetime.timedelta(500, 'ms')

In [ ]:
unique_timestamps[0] + np.timedelta64(500, 'ms')

In [ ]:
np.timedelta64(20, 'D')

In [ ]:
np.unique(counts)

In [ ]:
time_dt = 1 / (counts + 1)
time_dt

In [ ]:
adjusted_timestamps = []

for base_timestamp, count in zip(unique_timestamps, counts):
    time_dt = 1000 / (count)
    
    current_dt = time_dt / 2
    for i in range(count):
        adjusted_timestamps.append(base_timestamp + np.timedelta64(int(current_dt), 'ms'))
        current_dt += time_dt

In [ ]:
adjusted_timestamps = np.array(adjusted_timestamps).astype('datetime64[ms]').astype('uint64')

In [ ]:
def create_interpolator(adjusted_timestamps, data):
    interpolation = scipy.interpolate.interp1d(adjusted_timestamps, data)
    def interpolator(timestamp):
        timestamp = np.datetime64(timestamp).astype('datetime64[ms]').astype('uint64')
        return interpolation(timestamp)
    
    return interpolator

keys = ['Gantry', 'Collimator', 'MLC distance at CRA', 'Jaw distance']

interpolators = {
    key: create_interpolator(adjusted_timestamps, icom_data['Gantry'])
    for key in keys
}

In [ ]:
interpolators['Gantry']('2020-01-16 17:00:33.500')

In [ ]:
np.datetime64('2020-01-16 17:57:29').astype('uint64')

In [ ]:
gantry_interp(np.datetime64('2020-01-16 17:57:29').astype('datetime64[ms]'))

In [ ]:
np.datetime64('2020-01-16 17:57:29').astype('uint64')

In [ ]:
int_timestamps